---

# Proj1: Array Methods Trigger algorithms

This is a project for the Course **ProSeisSN**. It deals with real data and uses the Course's Github Repos.

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***
#### The code $\Downarrow$ BELOW $\Downarrow$ runs a notebook with other dependencies

In [ ]:
#------ Import Libraries
import sys
import os
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
import matplotlib.dates as dates
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)

#------ Magic commands
%matplotlib inline
%matplotlib widget
#%pylab notebook

---
## Guide
Esse é um guia para a realização de um trabalho do curso **ProSeisSN** tendo como base dados reais e os Jupter Notebooks do Repositório 
` https://github.com/jandyr/ProSeisSN_Nbk `
- Sugere-se um trabalho conciso, que apresente apenas os resultados mais importantes, evitando expor material que não venha a acrescentar conteúdo relevante. Como regra geral recomenda-se classificar o material a ser incluído no texto entre: pouco, médio e muito relevante. O material julgado como pouco relevante não deve ser incluído, recomenda-se refletir antes de incluir um material de relevância média, somente incluir o que realmente contribuir à compreenção do texto.
- Recomenda-se o uso de linguagem concisa e direta, evitando conceitos vazios do tipo: ''a partir dos dados``, ''vemos que`` etc. Tanto o Português, quanto o Inglês são aceitáveis na redação do texto.
- Provê-se um template em Latex **Proj1.tex** que *pode, ou não* ser utilizado. a escolha é sua.


---
## Use data files from TTB22
Os dados vem de um experimento realizado no Rio Pará, o braço oriental do estuário do Rio Amazonas, onde as ondas sísmicas são geradas pela transferência da energia do fluxo turbulento, resultante das grandes amplitudes de suas marés, por forças friccionais às rugosidades do seu leito. Tanto as fontes, quanto os geofones estão na superfície, tornando as ondas de superfície dominantes em relação às de volume.
- **The Receiver Array**
The array has 24 GS-20DX vertical geophones hooked to a $L=69$m cable, using takeouts spaced $\delta l=6$m from each other. The GS-20DX geophones have a natural frequency of $f_n=10\textrm{Hz}$, and a spurious frequency $f_{sp}>250$Hz. The array has a irregular circular shape, deployed in the Southern tip of the island, with its center at $\left(1^{\circ}12^{\prime}6.93^{\prime\prime}\textrm{S},48^{\circ}30^{\prime}23.39^{\prime\prime}\textrm{S}\right)$. Each of the 24 gather traces are $\Delta T=60$s long, with a sampling frequency of $f_{s}=250$Hz.

In [ ]:
#
"""
====================== READ PHONES LOCATIONS ======================
"""
#------ Read the phones cartesian locations
#--- Reads the CSV file with (x, y)m locations
ttb_loc = u.RGloc('../Data/'+'ttb_loc.dat')
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb_gloc = u.RGloc('../Data/'+'ttb_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb_loc[:,1], ttb_loc[:,2], ttb_loc[:,0], coord='cartesian')

---
## Read data files from TTB22
- **The data**
Work with 116 files, from 3695 to 3810, 2h of data em maré vazante variando 0.91m:
| File | Timestamp | Tide (m) |
| :-: | :-: | :-: |
| 3695 | 2022-04-02T12:00:15 | 2.70 |
| 3810 | 2022-04-02T14:03:06 | 1.79 |

<img src="./ttbtide.png" width="500">
#### Repeat the code $\Downarrow$ BELOW $\Downarrow$ as may times as needed

In [ ]:
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
"""
#------ Read the seismic data
ent = str(np.random.randint(3695, 3810))
ent = input(f'   Enter a file number in [3695, 3810], rtn=random:\n') or ent
ent = ent.rstrip().split(' ')
print(f">> Read with data file {ent}")
ent = '../Data/ttb22/'+ent[0]+'.dat'
#
#------- Read the data file as a SEG2 object

print(ent)
st = read(ent)
#
#------- Print stream information
dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f">> Gather acquired on {st[0].stats.starttime}, has {int(st[0].stats.npts)} data points.")

---
## Display the seismogram
- Use the experience with one trace to process the whole stream together.
- Try other MTparam

In [ ]:
#
#------ Create a new stream from the SEG2 stream. Gather baricenter = bcenter
gather, bcenter = u.creastrm(st, ttb_gloc)
#------- Filter the stream
#--- Filter parameters
MTparam = [1, 1, 'bp',  5., 50., 1, 0]
# └─────> [dtr, line, ftype, Fmin, Fmax, taper, gain]
gather = u.otrstr(gather, MTparam)
#
#------ Plot
gather.plot(type='section',
            scale=1.3, alpha=.7,
            orientation='horizontal')

#### $\Downarrow$ Zoom in the above seismogram $\Downarrow$

In [ ]:
#
#------ Zoom in the seismogram
ent = input(f' Enter t0 and t1 to zoom: ')
ent = ent.rstrip().split(' ')
f0 = float(ent[0])
f1 = float(ent[1])
#
dt = gather[0].stats.starttime
gather.plot(type='section',
            scale=1.3, alpha=.7,
            starttime=dt+f0, endtime=dt+f1,
            orientation='horizontal')

---
## Matched-filter detection
- Create a catalog of **template waveforms** from file 3804 to generate characteristic functions that could be used for detection.
- Once the catalog is created it is necessary to provide two paramters:
1) The correlation threshold within the normalised correlation range [-1, 1]
2)  The triger off time in s. This is the minimum distance between events.
#### The code $\Downarrow$ BELOW $\Downarrow$ resets the event catalog to an empty list: it should be run at least once.

In [ ]:
#------ Reset event catalog
cat   = [] #-> wavefrom catalog
tname = [] #-> template names
print(f">> Event catalog reset")

#### The code $\Downarrow$ BELOW $\Downarrow$ can be run many times to build the catalog

In [ ]:
"""
Build a catalog of templates write to file wvfrm.dat
"""
#------ Construct catalog
#------ Pick up a phone
dummy = np.random.randint(1, len(st)+1)
ent = input(f' Enter a phone [dflt=random] ')
ent = int( ent.rstrip().split(' ')[0] ) if ent else dummy
#------- The trace = Phone -1.
trZ = gather[ent - 1].copy()
#
#-------- Construct catalog of template waveforms with names
cat, tname = u.cat_wfrm(trZ, cat, tname, ent)
#
print(f">> Template catalog has {len(cat)} templates: {tname}")

---
- Save your work when finished.


In [ ]:
import json
"""
Save catalog to file name.wvf
""" 
ent = 'cat'
ent = input(f' Enter filename [dflt=cat.wvf] ') or ent
ent = ent.rstrip().split(' ')
ent = '../Data/'+ent+'.wvf'
with open(ent, 'w') as file:
    json.dump(data, file
print(f"wavefroms saved to {ent}")

---
## Detect events with the Matched-filter detection algorithms
Work with a single trace. It is necessary to provide two paramters:
- The correlation threshold within the normalised correlation range [-1, 1]
- The triger off time in s. This is the minimum distance between events.
- Ref: https://docs.obspy.org/master/tutorial/code_snippets/xcorr_detector.html

In [ ]:
#------ Import Packages
from obspy.signal.cross_correlation import correlation_detector
import pprint
print(f">> Trigger packages imported.")
"""
You can read the catalog file here if you want
""" 
#
#------ Enter height and distance (s). Hint: .6 1
ent = input(f'<< Enter correlation threshold and triger off time(s): ')
ent = ent.rstrip().split(' ')
corthr   = float(ent[0])
trgoff   = float(ent[1])
#
stream = Stream()
for i in range(0, len(gather), 6):
    stream  = Stream([gather[i]])
#
#------ Detections. Free memory on each loop.
    detect = []
    for template in cat:
        tmplstr = Stream([template])
        dets, sims = correlation_detector(
                       stream=stream, templates=[tmplstr],
                       heights=corthr, distance=trgoff, plot=stream,
                       template_names=tname )
        detect.extend(dets)
#
    print("Trace {0} has {1} detections from {2} templates".format(i+1, len(detect), len(cat)))
    print(detect)

---
